In [5]:
import os

if not os.path.exists("templates"):
    os.makedirs("templates")

html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Credit Card Segmentation</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background: #0d1117;
            color: #e6edf3;
        }
        .container {
            max-width: 750px;
            margin: 40px auto;
            padding: 30px;
            background: #161b22;
            border-radius: 15px;
            box-shadow: 0 8px 25px rgba(0,0,0,0.6);
            animation: fadeIn 1s ease-in-out;
        }
        h1 {
            text-align: center;
            font-size: 1.6rem;
            margin-bottom: 25px;
            color: #00b4d8;
            font-weight: bold;
            letter-spacing: 0.5px;
        }
        label {
            display: block;
            margin: 12px 0 6px;
            font-weight: 500;
        }
        input {
            width: 100%;
            padding: 12px;
            border: none;
            border-radius: 8px;
            background: #21262d;
            color: #e6edf3;
            outline: none;
            transition: all 0.2s;
        }
        input:focus {
            background: #2d333b;
            box-shadow: 0 0 0 2px #00b4d8;
        }
        button {
            margin-top: 20px;
            width: 100%;
            padding: 12px;
            border: none;
            border-radius: 8px;
            background: linear-gradient(90deg, #00b4d8, #0077b6);
            color: white;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: transform 0.2s, background 0.3s;
        }
        button:hover {
            transform: translateY(-2px);
            background: linear-gradient(90deg, #0096c7, #005f73);
        }
        #result {
            margin-top: 25px;
            padding: 15px;
            background: #21262d;
            border-radius: 10px;
            text-align: center;
            font-size: 1.1rem;
            color: #90e0ef;
            animation: slideUp 0.8s ease-in-out;
        }
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }
        @keyframes slideUp {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Unsupervised Learning for Credit Card Customer Segmentation</h1>
        <form id="predictForm">

            <label>Balance:</label>
            <input type="number" step="any" name="BALANCE" required>

            <label>Balance Frequency:</label>
            <input type="number" step="any" name="BALANCE_FREQUENCY" required>

            <label>Purchases:</label>
            <input type="number" step="any" name="PURCHASES" required>

            <label>One-off Purchases:</label>
            <input type="number" step="any" name="ONEOFF_PURCHASES" required>

            <label>Installments Purchases:</label>
            <input type="number" step="any" name="INSTALLMENTS_PURCHASES" required>

            <label>Cash Advance:</label>
            <input type="number" step="any" name="CASH_ADVANCE" required>

            <label>Purchases Frequency:</label>
            <input type="number" step="any" name="PURCHASES_FREQUENCY" required>

            <label>One-off Purchases Frequency:</label>
            <input type="number" step="any" name="ONEOFF_PURCHASES_FREQUENCY" required>

            <label>Purchases Installments Frequency:</label>
            <input type="number" step="any" name="PURCHASES_INSTALLMENTS_FREQUENCY" required>

            <label>Cash Advance Frequency:</label>
            <input type="number" step="any" name="CASH_ADVANCE_FREQUENCY" required>

            <label>Cash Advance Transactions:</label>
            <input type="number" step="any" name="CASH_ADVANCE_TRX" required>

            <label>Purchases Transactions:</label>
            <input type="number" step="any" name="PURCHASES_TRX" required>

            <label>Credit Limit:</label>
            <input type="number" step="any" name="CREDIT_LIMIT" required>

            <label>Payments:</label>
            <input type="number" step="any" name="PAYMENTS" required>

            <label>Minimum Payments:</label>
            <input type="number" step="any" name="MINIMUM_PAYMENTS" required>

            <label>% Full Payment:</label>
            <input type="number" step="any" name="PRC_FULL_PAYMENT" required>

            <label>Tenure:</label>
            <input type="number" step="any" name="TENURE" required>

            <button type="button" onclick="predict()">Predict</button>
        </form>

        <div id="result">
            <p id="prediction"></p>
        </div>
    </div>

    <script>
        async function predict() {
            const form = document.getElementById("predictForm");
            const formData = new FormData(form);

            const response = await fetch("/predict", {
                method: "POST",
                body: formData
            });

            const result = await response.json();
            document.getElementById("prediction").innerText =
                "Customer Segment: " + (result.cluster_label || result.error);
        }
    </script>
</body>
</html>
'''

with open("templates/index.html", "w") as f:
    f.write(html_template)


In [6]:
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import joblib
import pandas as pd
import numpy as np

# Load saved objects
scaler = joblib.load("minmax_scaler_credit.pkl")
pca = joblib.load("pca_transformer.pkl")
kmeans = joblib.load("best_kmeans_model.pkl")

# Cluster interpretation
cluster_mapping = {
    0: "Low Spender",
    1: "High Spender"
}

# Flask app
app = Flask(__name__)

# Setup ngrok
ngrok.set_auth_token("your_token")  # replace with your ngrok token
port = 5001
public_url = ngrok.connect(port)
print(f"🌍 Ngrok URL: {public_url}")

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Collect all inputs
        input_data = {key: float(request.form[key]) for key in request.form.keys()}
        input_df = pd.DataFrame([input_data])

        # Apply log1p transform to skewed columns
        skewed_cols = ["BALANCE", "PURCHASES", "ONEOFF_PURCHASES",
                       "INSTALLMENTS_PURCHASES", "CASH_ADVANCE",
                       "PAYMENTS", "MINIMUM_PAYMENTS"]

        for col in skewed_cols:
            input_df[col + "_log"] = np.log1p(input_df[col])

        # Drop raw skewed columns (to match training df_clean)
        input_df_clean = input_df.drop(columns=skewed_cols)

        # Scale
        scaled = scaler.transform(input_df_clean)

        # PCA
        reduced = pca.transform(scaled)

        # Predict cluster
        cluster = kmeans.predict(reduced)[0]
        cluster_label = cluster_mapping.get(cluster, "Unknown Segment")

        return jsonify({"cluster": int(cluster), "cluster_label": cluster_label})

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run(port=port)


🌍 Ngrok URL: NgrokTunnel: "https://0e09aae1efe5.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [07/Sep/2025 15:53:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2025 15:53:23] "GET /favicon.ico HTTP/1.1" 404 -
E:\anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
127.0.0.1 - - [07/Sep/2025 15:54:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2025 15:54:52] "GET / HTTP/1.1" 200 -
E:\anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
127.0.0.1 - - [07/Sep/2025 15:56:44] "POST /predict HTTP/1.1" 200 -
